In [8]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import webbrowser
import os
from IPython.display import display, HTML  # For displaying HTML in Jupyter (optional)

# File path
file_path = r"D:\John MacTavish\Solar Data\Solar Flare .xlsx"

# Load data from Excel
try:
    df = pd.read_excel(file_path)
    print("Data loaded successfully!")
except Exception as e:
    print(f"Error loading file: {str(e)}")
    raise

# Ensure column names match the data
expected_columns = ['Date', 'Number of Sunspots', 'Class Magnitude', 'Class Spot']
if not all(col in df.columns for col in expected_columns):
    print("Error: Excel file must contain 'Date', 'Number of Sunspots', 'Class Magnitude', and 'Class Spot' columns.")
    raise

# Data preprocessing
df['Date'] = pd.to_datetime(df['Date'])
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year

# Encode categorical variables with new classification
df['Magnitude_Encoded'] = df['Class Magnitude'].map({'B': 1, 'C': 1, 'M': 2, 'X': 3})  # B and C combined as Alpha (1), M as Beta (2), X as Gamma (3)
spot_class_map = {cls: idx for idx, cls in enumerate(df['Class Spot'].unique())}
df['Spot_Class_Encoded'] = df['Class Spot'].map(spot_class_map)

# Prepare features and target
features = ['Number of Sunspots', 'Day', 'Month', 'Year', 'Spot_Class_Encoded']
X = df[features]
y = df['Magnitude_Encoded']

# Split data and train model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Function to get historical averages
def get_historical_averages(month, year):
    similar_data = df[df['Month'] == month]
    if similar_data.empty:
        similar_data = df
    avg_sunspots = int(similar_data['Number of Sunspots'].mean())
    most_common_spot = similar_data['Class Spot'].mode()[0]
    spot_class_encoded = spot_class_map.get(most_common_spot, 0)
    return avg_sunspots, most_common_spot, spot_class_encoded

# Prediction function with HTML output matching the image size
def predict_solar_flare(date_str):
    try:
        date = datetime.strptime(date_str, '%Y-%m-%d')
        avg_sunspots, spot_class, spot_class_encoded = get_historical_averages(date.month, date.year)
        
        input_data = np.array([[avg_sunspots, date.day, date.month, date.year, spot_class_encoded]])
        probabilities = model.predict_proba(input_data)[0]
        
        # Adjust probabilities for new classes
        class_probs = {
            'Alpha': (probabilities[0]) * 100,  # B and C combined
            'Beta': probabilities[1] * 100,     # M
            'Gamma': probabilities[2] * 100     # X
        }
        
        max_prob_class = max(class_probs, key=class_probs.get)
        if max_prob_class == 'Gamma':
            warning = "HIGH WARNING: Strong flares possible!"
            warning_color = "#fb8500"  # Orange
        elif max_prob_class == 'Beta':
            warning = "MODERATE WARNING: Moderate activity."
            warning_color = "#ffb703"  # Yellow
        else:
            warning = "LOW WARNING: Minimal risk."
            warning_color = "#219ebc"  # Teal

        # HTML content matching the image size and layout
        html_content = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Solar Flare - {date_str}</title>
            <style>
                body {{
                    font-family: 'Segoe UI', sans-serif;
                    background: linear-gradient(135deg, #0d1b2a, #1b263b);
                    color: #e0e0e0;
                    margin: 0;
                    padding: 15px;
                    display: flex;
                    justify-content: center;
                    min-height: 100vh;
                }}
                .container {{
                    background: rgba(255, 255, 255, 0.05);
                    border-radius: 10px;
                    padding: 20px;
                    max-width: 400px;
                    box-shadow: 0 5px 20px rgba(0, 0, 0, 0.3);
                    text-align: center;
                }}
                h1 {{
                    color: #ffb703;
                    font-size: 1.8em;
                    margin-bottom: 15px;
                    text-transform: uppercase;
                }}
                .prediction {{
                    font-size: 1em;
                    margin: 10px 0;
                }}
                .prediction p {{
                    margin-bottom: 10px;
                }}
                .warning {{
                    font-weight: 600;
                    padding: 10px;
                    border-radius: 8px;
                    margin: 10px 0;
                    background: {warning_color};
                    color: #023047;
                    text-transform: uppercase;
                }}
                .bar {{
                    width: 100%;
                    background: rgba(255, 255, 255, 0.1);
                    border-radius: 5px;
                    margin: 5px 0;
                    height: 15px;
                    overflow: hidden;
                }}
                .fill {{
                    height: 100%;
                    background: linear-gradient(to right, #219ebc, #8ecae6);
                    text-align: right;
                    padding-right: 5px;
                    color: #023047;
                    font-size: 0.8em;
                }}
                .info-box {{
                    background: rgba(255, 255, 255, 0.07);
                    padding: 15px;
                    border-radius: 10px;
                    margin-top: 15px;
                    font-size: 0.9em;
                    text-align: left;
                }}
                .info-box h3 {{
                    color: #ffb703;
                    font-size: 1.2em;
                    margin-bottom: 10px;
                    text-transform: uppercase;
                }}
                .info-box ul {{
                    padding-left: 15px;
                    list-style-type: none;
                }}
                .info-box ul li {{
                    margin: 5px 0;
                    position: relative;
                }}
                .info-box ul li:before {{
                    content: '☀️';
                    position: absolute;
                    left: -15px;
                }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1>Solar Flare Prediction for {date_str}</h1>
                <div class="prediction">
                    <p>Expected Sunspots: {avg_sunspots}</p>
                    <p>Chances of Solar Flare Strength:</p>
                    <div>Weak (Alpha): {class_probs['Alpha']:.1f}%</div>
                    <div class="bar"><div class="fill" style="width: {class_probs['Alpha']}%;">{class_probs['Alpha']:.1f}%</div></div>
                    <div>Moderate (Beta): {class_probs['Beta']:.1f}%</div>
                    <div class="bar"><div class="fill" style="width: {class_probs['Beta']}%;">{class_probs['Beta']:.1f}%</div></div>
                    <div>Strong (Gamma): {class_probs['Gamma']:.1f}%</div>
                    <div class="bar"><div class="fill" style="width: {class_probs['Gamma']}%;">{class_probs['Gamma']:.1f}%</div></div>
                </div>
                <div class="warning">{warning}</div>
                <div class="info-box">
                    <h3>What Are Solar Flares?</h3>
                    <p>Solar flares are sudden bursts of energy from the Sun. They can affect Earth in different ways:</p>
                    <ul>
                        <li><b>Weak (Alpha):</b> Usually harmless, minor radio noise possible (includes B and C classes).</li>
                        <li><b>Moderate (Beta):</b> Can cause radio blackouts in polar regions and minor satellite issues (M class).</li>
                        <li><b>Strong (Gamma):</b> May disrupt GPS, radio communications, and power grids, and create stunning auroras (X class)!</li>
                    </ul>
                    <p>Predictions are based on historical solar data and may vary.</p>
                </div>
            </div>
        </body>
        </html>
        """

        # Save HTML to file with UTF-8 encoding
        html_file = f"solar_flare_prediction_{date_str}.html"
        with open(html_file, "w", encoding='utf-8') as f:
            f.write(html_content)

        # Open in default browser
        webbrowser.open('file://' + os.path.realpath(html_file))
        
        # Optional: Display HTML inline in Jupyter (comment out if not needed)
        # display(HTML(html_content))
        
    except Exception as e:
        print(f"Error in prediction: {str(e)}")

# User input function
def get_user_date():
    while True:
        date_str = input("Enter the date you want to predict for (YYYY-MM-DD, e.g., 2025-03-22): ")
        try:
            datetime.strptime(date_str, '%Y-%m-%d')
            return date_str
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD (e.g., 2025-03-22).")

# Main execution
print("Welcome to the Solar Flare Prediction Tool!")
print("===========================================")
print("Enter a date to see the predicted chance of solar flares.")
print("A webpage will open with your results!\n")

date_str = get_user_date()
predict_solar_flare(date_str)

Data loaded successfully!
Welcome to the Solar Flare Prediction Tool!
Enter a date to see the predicted chance of solar flares.
A webpage will open with your results!

Enter the date you want to predict for (YYYY-MM-DD, e.g., 2025-03-22): 2025-03-26


C:\Users\Asus\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
